Uptrail Week 1 project

In [395]:
import pandas as pd
import numpy as np

In [396]:
df = pd.read_csv("C:/Users/HP/Desktop/Uptrail/Week1 Dataset/customer_signups.csv")
df.head(5)

,customer_id,name,email,signup_date,source,region,plan_selected,marketing_opt_in,age,gender
0,CUST00000,Joshua Bryant,NaN,NaN,Instagram,NaN,basic,No,34,Female
1,CUST00001,Nicole Stewart,nicole1@example.com,02-01-2024,LinkedIn,West,basic,Yes,29,Male
2,CUST00002,Rachel Allen,rachel2@example.com,03-01-2024,Google,North,PREMIUM,Yes,34,Non-Binary
3,CUST00003,Zachary Sanchez,zachary3@mailhub.org,04-01-2024,YouTube,NaN,Pro,No,40,Male
4,CUST00004,NaN,matthew4@mailhub.org,05-01-2024,LinkedIn,West,Premium,No,25,Other


In [397]:
df.count()

customer_id         298
name                291
email               266
signup_date         298
source              291
region              270
plan_selected       292
marketing_opt_in    290
age                 288
gender              292
dtype: int64

In [398]:
df.dtypes

customer_id         object
name                object
email               object
signup_date         object
source              object
region              object
plan_selected       object
marketing_opt_in    object
age                 object
gender              object
dtype: object

In [399]:
# df['signup_date'] = pd.to_datetime(df['signup_date'])
# df['signup_date'] = pd.to_datetime(df['signup_date'], dayfirst=True, errors='coerce')
# df.dtypes


In [400]:
# bad_values = ['not a date', '', 'n/a']
# df['signup_date'] = df['signup_date'].replace(bad_values, np.nan)
# df['signup_date'] = pd.to_datetime(df['signup_date'], errors='coerce')
# # df['signup_date'].count()
# df['signup_date'] = pd.to_datetime(df['signup_date'])
df['signup_date'] = pd.to_datetime(df['signup_date'], dayfirst=True, errors='coerce')
# df['signup_date'] = df['signup_date'].replace('', pd.NA)
median_date = df['signup_date'].median()
df['signup_date'] = df['signup_date'].fillna(median_date)


# df['signup_date'].count()

# df['signup_date'] = pd.to_datetime(df['signup_date'], dayfirst=True, errors='coerce')
print("Valid dates:", df['signup_date'].notna().sum())
print("Invalid (NaT) dates:", df['signup_date'].isna().sum())


Valid dates: 300
Invalid (NaT) dates: 0


In [401]:
# Show rows that failed parsing
df['signup_date'].head(85)


0    2024-05-30
1    2024-01-02
2    2024-01-03
3    2024-01-04
4    2024-01-05
        ...    
80   2024-05-30
81   2024-03-22
82   2024-03-23
83   2024-03-24
84   2024-03-25
Name: signup_date, Length: 85, dtype: datetime64[ns]

In [402]:
#converting marketing-opt_in column into boolean data type and handling missing values
df['marketing_opt_in'] = df['marketing_opt_in'].astype(str).str.lower().str.strip()
df['marketing_opt_in'] = df['marketing_opt_in'].map({
    'yes': True,
    'no' : False
})
df['marketing_opt_in'] = df['marketing_opt_in'].fillna(False)


In [403]:
#checking for unique values in the column
df['marketing_opt_in'].unique()

array([False,  True])

In [404]:
#checking for null values in the column 
df['marketing_opt_in'].isna().sum()

0

In [405]:
#cleaning customer_id column 
#cheking for null values
df['customer_id'].isna().sum()

2

In [406]:
#extracting numeric part and fillinh null cells as they are in sequence
df['customer_id_numeric'] = df['customer_id'].str.extract(r'(\d{5})')
df['customer_id_numeric'] = pd.to_numeric(df['customer_id_numeric'], errors='coerce')
df['customer_id_numeric'] = df['customer_id_numeric'].interpolate(method='linear')
df['customer_id_numeric'] = df['customer_id_numeric'].round().astype(int)
df['customer_id'] = df['customer_id_numeric'].apply(lambda x: f"CUST{x:05d}")

# Step 5: Drop helper column
# df.drop(columns='customer_id_numeric', inplace=True)


In [407]:
#double chaecking for null values
df['customer_id'].isna().sum()


0

In [408]:
#Dropping helper column
df.drop(columns='customer_id_numeric', inplace=True)

In [409]:
# Handling missing name where email is available
def infer_name_from_email(email):
    if pd.isna(email):
        return np.nan
    username = email.split('@')[0]
    name_parts = ''.join([c if not c.isdigit() else ' ' + c for c in username]).split()
    return name_parts[0].capitalize() if name_parts else np.nan

df['name'] = df.apply(
    lambda row: infer_name_from_email(row['email']) if pd.isna(row['name']) else row['name'],
    axis=1
)

#fill null cells as not specified
df['name'] = df['name'].fillna('Not Specified')



In [410]:
#checking for the null values after filling names from email 
# df['name'].isna().sum()
df['name'].count()


300

In [411]:
#checking for null values in the email column
df['email'].isna().sum()

34

In [412]:
#converting null vlaues as not specified
df['email'] = df['email'].replace(r'^\s*$', np.nan, regex=True)  
df['email'] = df['email'].fillna('Not Specified')                

print(df['email'].isna().sum())


0


In [413]:
#cleaning source column by replacing null values by Unkown 
df['source'] = df['source'].replace(r'^\s*$|^\?\?$', np.nan, regex=True)

df['source'] = df['source'].fillna('Unknown')

df['source'] = df['source'].str.title()


In [414]:
df['source'].unique()

array(['Instagram', 'Linkedin', 'Google', 'Youtube', 'Facebook',
       'Referral', 'Unknown'], dtype=object)

In [415]:
df['source'].isna().sum()

0

In [416]:
#cleaning region column
#cheking for null and unique vlaues
df['region'].unique()

array([nan, 'West', 'North', 'South', 'Central', 'East'], dtype=object)

In [417]:
print(df['region'].count())
print(df['region'].isna().sum())

270
30


In [418]:
# Clean 'region' column
df['region'] = df['region'].replace(r'^\s*$', np.nan, regex=True)  
df['region'] = df['region'].fillna('Unknown')                      
df['region'] = df['region'].str.title()                            


In [419]:
#rechecking the region column
print(df['region'].count())
print(df['region'].isna().sum())
print(df['region'].unique())

300
0
['Unknown' 'West' 'North' 'South' 'Central' 'East']


In [420]:
#cleaning plan_selected Column
df['plan_selected'].unique()

array(['basic', 'PREMIUM', 'Pro', 'Premium', 'UnknownPlan', 'PRO',
       'Basic', nan, 'prem'], dtype=object)

In [421]:
print(df['plan_selected'].isna().sum())
print(df['plan_selected'].value_counts(dropna=True))
print(df['plan_selected'].count())

8
plan_selected
Premium        57
Pro            53
basic          46
Basic          46
PREMIUM        42
PRO            41
UnknownPlan     6
prem            1
Name: count, dtype: int64
292


In [422]:
# dealing with null values
df['plan_selected'] = df['plan_selected'].replace(r'^\s*$', np.nan, regex=True)
df['plan_selected'] = df['plan_selected'].fillna('UnknownPlan')

# Converting all to lowercase
df['plan_selected'] = df['plan_selected'].str.lower()

# column integration/ quality
df['plan_selected'] = df['plan_selected'].replace({
    'prem': 'premium'  
})


# df['plan_selected'] = df['plan_selected'].astype('category')


In [423]:
print(df['plan_selected'].isna().sum())
print(df['plan_selected'].value_counts(dropna=True))
print(df['plan_selected'].count())

0
plan_selected
premium        100
pro             94
basic           92
unknownplan     14
Name: count, dtype: int64
300


In [424]:
#cleaning age column 
df['age'].unique()

array(['34', '29', '40', '25', '60', '47', '53', '21', nan, 'unknown',
       'thirty', '206'], dtype=object)

In [425]:
print(df['age'].isna().sum())
print(df['age'].value_counts(dropna=True))
print(df['age'].count())

12
age
40         50
25         49
29         46
34         44
21         28
53         27
47         22
60         14
unknown     6
thirty      1
206         1
Name: count, dtype: int64
288


In [426]:
# Replacing string 'thirty' with 30 and 'unknown' with np.nan
df['age'] = df['age'].replace('thirty', 30)
df['age'] = df['age'].replace('unknown', np.nan)

# # Convert column to numeric, forcing errors to NaN
df['age'] = pd.to_numeric(df['age'], errors='coerce')


# Remove the outlier 206 (assuming it's a data entry error)
df.loc[df['age'] == 206, 'age'] = np.nan

# Calculate the median after cleaning
median_age = df['age'].median()

# Fill NaN values (original NaNs, 'unknown', 'thirty' conversion issues, and 206) with median
df['age'] = df['age'].fillna(median_age)

# Optional: Convert to integer if needed
df['age'] = df['age'].astype(int)

# Final check
print(median_age)
print(df['age'].isna().sum())      
print(df['age'].value_counts())   


34.0
0
age
34    63
40    50
25    49
29    46
21    28
53    27
47    22
60    14
30     1
Name: count, dtype: int64


In [427]:
#cleaning gender column
df['gender'].unique()

array(['Female', 'Male', 'Non-Binary', 'Other', 'male', 'FEMALE', nan,
       '123'], dtype=object)

In [428]:
print(df['gender'].isna().sum())
print(df['gender'].value_counts(dropna=True))
print(df['gender'].count())

8
gender
Other         59
FEMALE        52
male          49
Male          44
Non-Binary    42
Female        41
123            5
Name: count, dtype: int64
292


In [429]:
# Standardizing case to title case
df['gender'] = df['gender'].str.title()

# Replacing '123' and NaNs with 'Other'
df['gender'] = df['gender'].replace(['123', np.nan], 'Other')

# Validating and restricting to known categories
valid_genders = ['Male', 'Female', 'Non-Binary', 'Other']
df['gender'] = df['gender'].apply(lambda x: x if x in valid_genders else 'Other')

# Final check
print(df['gender'].isna().sum())            
print(df['gender'].value_counts())          


0
gender
Female        93
Male          93
Other         72
Non-Binary    42
Name: count, dtype: int64


In [430]:
df.dtypes

customer_id                 object
name                        object
email                       object
signup_date         datetime64[ns]
source                      object
region                      object
plan_selected               object
marketing_opt_in              bool
age                          int32
gender                      object
dtype: object

In [431]:
# Converting appropriate columns to 'category'
cat_cols = ['source', 'region', 'plan_selected', 'gender']
df[cat_cols] = df[cat_cols].astype('category')
df.dtypes


customer_id                 object
name                        object
email                       object
signup_date         datetime64[ns]
source                    category
region                    category
plan_selected             category
marketing_opt_in              bool
age                          int32
gender                    category
dtype: object

In [432]:
df.value_counts(dropna=False)

customer_id  name               email                   signup_date  source     region   plan_selected  marketing_opt_in  age  gender    
CUST00000    Joshua Bryant      Not Specified           2024-05-30   Instagram  Unknown  basic          False             34   Female        1
CUST00206    Todd Peck III      todd6@example.com       2024-07-25   Unknown    East     pro            False             25   Non-Binary    1
CUST00204    Patricia Powers    patricia4@mailhub.org   2024-07-23   Referral   North    basic          False             34   Other         1
CUST00203    Kevin Brown        kevin3@mailhub.org      2024-07-22   Instagram  West     basic          False             29   Female        1
CUST00202    Donald Underwood   donald2@example.com     2024-07-21   Referral   West     basic          True              40   Female        1
                                                                                                                                            ..
CUST

In [433]:
df.isna().sum()

customer_id         0
name                0
email               0
signup_date         0
source              0
region              0
plan_selected       0
marketing_opt_in    0
age                 0
gender              0
dtype: int64